# Heading toward parallelism


## Built-in Functions that we haven't covered 

- enumerate()
- filter()
- map()
- zip()
- lambda functions

### Enumerate

Return item and item number

In [1]:
class axis:
    def __init__(self,n,o=0.,d=1.):
        self.n=n
        self.o=o
        self.d=d

axes=[axis(4),axis(3,1.),axis(5,0.,2.)]

for iax,ax in enumerate(axes):
    print(iax,ax.n)

0 4
1 3
2 5


### Filter

Filter a list based on a functon

In [2]:
numbers = [70, 60, 80, 90, 50,82,90,91,84,82,94,99,78,65,61,45,89,87,49,76,81,94]
def not_divisible_10(n):
    return n%10!=0
print(list(filter(not_divisible_10, numbers)))


[82, 91, 84, 82, 94, 99, 78, 65, 61, 45, 89, 87, 49, 76, 81, 94]


### lambda

A shorthand way to create a function

In [3]:
div=lambda x: x%10!=0
print(list(filter(div, numbers)))


[82, 91, 84, 82, 94, 99, 78, 65, 61, 45, 89, 87, 49, 76, 81, 94]


In [4]:
print(list(filter(lambda x: x%10!=0, numbers)))


[82, 91, 84, 82, 94, 99, 78, 65, 61, 45, 89, 87, 49, 76, 81, 94]


### map

Map a function to a list (or dictionary)

In [5]:
numbers = [1, 2, 3, 4, 5, 6]
squared_numbers = list(map(lambda x: x * x, numbers))

print(squared_numbers)

[1, 4, 9, 16, 25, 36]


### zip

From a series of list make a list of tuples


In [6]:
names = ['John', 'Jane', 'Jim']
ages = [30, 31, 32]
countries = ['USA', 'UK', 'Canada']

zip_result = list(zip(names, ages, countries))

print(zip_result)


[('John', 30, 'USA'), ('Jane', 31, 'UK'), ('Jim', 32, 'Canada')]


### Why???

- Single line functions are faster 
- Make you look like a "python pro" 
    - snob?

## Make a 2-D array from lists using a loop and on a single line

In [7]:
%%timeit
n,m=10000,10000
arr=[]
for i in range(n):
    arr1=[]
    for j in range(m):
        arr1.append(0)
    arr.append(arr1)

7.35 s ± 439 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
arr=[[0 for i in range (10000)] for j in range (10000)]

3.22 s ± 133 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Numpy

How about doing the same thing in numpy

In [9]:
import numpy as np

In [10]:
%%timeit
arr=np.zeros((10000,10000))

3.11 µs ± 18.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Appending even

In [11]:
%%timeit 

arr=[]
for i in range(100*1000*1000):
    if i%2==0:
        arr.append(i)

8.86 s ± 373 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Do the same thing one line

In [12]:
%%timeit
arr=[i for i in range(100*1000*1000) if i%2==0]

6.91 s ± 325 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Now in numpy with what amounts to a lambda function

In [13]:
%%timeit
ar = np.arange(100*1000*1000)
ar = ar[ar%2==0]


2.56 s ± 212 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Now if we do the same thing completely in numpy (C)

In [14]:
%%timeit
ar=np.arange(2,10000,2)

2.94 µs ± 29.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Numba

We've seen how fast numpy can be compared to python. Another option is numba.
Numba is a Just In Time (JIT) compiler.  When a numba decorated function is called
it looks at the arguments. If it hasn't compiled a version of the function with
those arguments it, it compiles the function.  It is very efficient numpy arrays,
not so efficient on python types.


In [15]:
import random
import numpy.random
ar1=[random.random() for i in range(100*1000*1000)]
ar2=[random.random() for i in range(100*1000*1000)]
ar3=numpy.random.rand(100*1000*1000)
ar4=numpy.random.rand(100*1000*1000)

In [16]:
import numba
@numba.jit()
def dot_it(a1,a2,n):
    tot=0
    for i in range(n):
        tot+=a1[i]*a2[i]
    return tot
#x=dot_it(ar1,ar2,100*1000*1000)
y=dot_it(ar3,ar4,100*1000*1000)

In [17]:
#%%timeit
#dot_it(ar1,ar2,100*1000*1000)

In [18]:
%%timeit
dot_it(ar3,ar4,100*1000*1000)

145 ms ± 6.96 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Don't send python base classes, use numpy types!

In [19]:
@numba.njit() #only run with non-python classes
def dotit(ar1,ar2):
    out=0
    for i in range(ar1.shape[0]):
        out+=ar2[i]*ar1[i]
    return out
print(dotit(ar3,ar4))


25001681.827542666


## Numba and loop parallelism
- Numba can parallelize loops (using omp behind the scenes)
- Limited parallelism model
    - Define loop with numba.prange
    - Can Do limited reduce operations

In [20]:
@numba.njit(parallel=True) #only run with non-python classes
def dotit(ar1,ar2):
    out=0
    for i in numba.prange(ar1.shape[0]):
        out+=ar2[i]*ar1[i]
    return out
print(dotit(ar3,ar4))

25001681.827537928


In [21]:
%%timeit 
y=dotit(ar3,ar4)

56.7 ms ± 2.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


How does it compare to direct numpy?

In [22]:
%%timeit
y=ar3.dot(ar4)

94.5 ms ± 3.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Why is numpy faster than the serial version (and sometimes the parllel version)?  Numpy is calling a highly optimized, vectorzied BLAS library. It is maximizing vector performance. Even using more cores doesn't help.

## Is strict numpy always the answer?

Lets change the problem to applying a laplacian to an array.

In [43]:
import numpy.random
import numpy as np
inp=numpy.random.rand(10000,10000)
outp=np.zeros((10000,10000))


The numpy way to write this operation looks something like the following:

In [37]:
%%timeit
outp[1:-1,1:-1]=inp[1:-1,1:-1]-.25*(inp[2:,1:-1]+inp[:-2,1:-1]+\
                                    inp[1:-1,2:]+inp[1:-1,:-2])

1.47 s ± 24.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
%%timeit
outp[1:-1,1:-1] -= inp[1:-1,1:-1]
outp[1:-1,1:-1] += inp[1:-1,:-2]
outp[1:-1,1:-1] += inp[1:-1,2:]
outp[1:-1,1:-1] += inp[:-2,1:-1]
outp[1:-1,1:-1] += inp[2:,1:-1]

1.04 s ± 11.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


For comparison the single line python way.

In [25]:
#%%timeit 
#out=[[inp[i,j]-.25*(inp[i-1,j]+inp[i+1,j]+inp[i,j-1]+inp[i,j+1]) for i in range (1,9999)] for j in range (1,9999)]

And the numba approach

In [38]:
import numba
@numba.njit()
def apply_stencil(inp,outp):
    for i in range(1,inp.shape[0]-1):
        for j in range(1,inp.shape[1]-1):
            outp[i,j]=inp[i,j]-.25*(inp[i-1,j]+inp[i+1,j]+inp[i,j-1]+inp[i,j+1])

In [39]:
#runit once to compile function
apply_stencil(inp,outp)

In [40]:
%%timeit
apply_stencil(inp,outp)

298 ms ± 38.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


And a parallelized numba approach

In [44]:
import numba
@numba.njit(parallel=True)
def apply_stencil_par(inp,outp):
    for i in range(1,inp.shape[0]-1):
        for j in numba.prange(1,inp.shape[1]-1):
            outp[i,j]=inp[i,j]-.25*(inp[i-1,j]+inp[i+1,j]+inp[i,j-1]+inp[i,j+1])
apply_stencil_par(inp,outp)

In [45]:
%%timeit
apply_stencil_par(inp,outp)

738 ms ± 32.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In this case we are getting a large speedup using numba. Why?
- The numpy is creating a bunch of extra arrays
- Destroys all cache reuse

What about scipy?

In [35]:
%%timeit
from scipy import ndimage
result = ndimage.laplace(inp)


4.09 s ± 481 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
